In [1]:
import requests
from bs4 import BeautifulSoup

In [ ]:
import time
import random
import pandas as pd

In [2]:
import sys
import os

In [3]:
def flushprint(www):
    sys.stdout.write('\r')
    sys.stdout.write('%s' %www)
    sys.stdout.flush()

In [5]:
# 1-城市（住宿类型、酒店名称、酒店link）.ipynb 保存的文件
hotel_list=pd.read_excel('beijing_link.xls',index_col=0)

In [6]:
len(hotel_list)

790

In [7]:
hotel_list.head()

,star-rating,name,link
0,1,北京有家国际青年旅舍,/hotel/cn/youjia-international-youth-beijing.z...
1,2,北京和园国际青年客栈,/hotel/cn/heyuan-international-youth-hostel.zh...
2,2,北京隆福天缘宾馆,/hotel/cn/happy-dragon-hostel.zh-cn.html?label...
3,2,来不轻旅丨三里屯店,/hotel/cn/sheng-sha-bin-guan.zh-cn.html?label=...
4,2,锦江之星北京广渠门店,/hotel/cn/jj-inns-beijing-guangqu-gate.zh-cn.h...


In [8]:
headers={
    'cookie':'用自己的啦',
    'user-agent':'用自己的啦'
}

In [8]:
hotel_score=[]
# hotel_error=[]#报错则将错误信息收集

In [13]:
for index in range(601,790): #hotel_list.index:
    #剔除href中的识别信息
    hotel_info_link=hotel_list.iloc[index]['link'].replace('&from=searchresults;highlight_room=#hotelTmpl','')
    url='https://www.booking.com'\
        '{link}'\
        '&#tab-reviews'.format(link=hotel_info_link)
    
    star_rating=hotel_list.iloc[index]['star-rating']#酒店住宿评级 
    name=hotel_list.iloc[index]['name']#酒店中文名称
    link=hotel_list.iloc[index]['link']
    
    try:
        tt = random.random()
        flushprint(str(url)+':sleeping'+str(tt)+'seconds')
        time.sleep(tt)
    
        #爬取酒店详情网页
        html=requests.get(url)
        html.encoding='utf-8'
        content=html.text
        soup=BeautifulSoup(content,'html.parser')

        if soup.find('div',class_='not_enough_reviews'):
            hotel=[star_rating,name,link,url,'NaN','NaN','NaN']
            hotel_score.append(hotel)
        else:
            score=soup.find('span',class_='review-score-badge').text.replace('\n','') #酒店评分
            score_title=soup.find('span',class_='review-score-widget__text').text #评分等级
            review_num=soup.find('span',class_='review-score-widget__subtext').text.replace('\n','')#评分人数
            hotel=[star_rating,name,link,url,score,score_title,review_num]
            hotel_score.append(hotel)
            
#         #评分细项
#         for item in soup.find_all('span',class_='c-score-bar__score'):
#             hotel.append(item.text)

    except Exception as e:
        print(index,url,e)
        error=[index,star_rating,name,link,url]
        hotel_error.append(error)
        continue

https://www.booking.com/hotel/cn/798yi-zhu-qu-lofthong-yuan-gong-yu-shang-xia-pu-wei-di-tie-14hao-xian-wang-jing-.zh-cn.html?label=gen173nr-1DCAQoggJCDXNlYXJjaF_ljJfkuqxIK1gEaDGIAQGYASu4AQfIAQzYAQPoAQH4AQOIAgGoAgO4Av-pzu8FwAIB&sid=7935495a3dd5961707c88bd0b2739902&all_sr_blocks=375992601_208915282_1_0_0%2C375992601_208915282_1_0_0&checkin=2020-01-10&checkout=2020-01-11&dest_id=-1898541&dest_type=city&group_adults=2&group_children=0&hapos=118&highlighted_blocks=375992601_208915282_1_0_0%2C375992601_208915282_1_0_0&hpos=18&nflt=class%3D0%3B&no_rooms=1&sr_order=popularity&srepoch=1576244479&srpvid=7682603f4954012b&ucfs=1&#tab-reviews:sleeping0.6250743556194123seconds

In [16]:
hotel_score_all=pd.DataFrame(hotel_score,columns=['star-rating','name','link','hotel-info-link','score','score-title','review-num'])

In [17]:
hotel_score_all.to_excel('beijing_score.xls')